<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BERT" data-toc-modified-id="BERT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BERT</a></span><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-LGBMClassifier" data-toc-modified-id="Модель-LGBMClassifier-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Модель LGBMClassifier</a></span></li><li><span><a href="#Модель-RandomForestClassifier" data-toc-modified-id="Модель-RandomForestClassifier-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Модель RandomForestClassifier</a></span></li><li><span><a href="#Тестирование-наилучшей-модели" data-toc-modified-id="Тестирование-наилучшей-модели-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Тестирование наилучшей модели</a></span></li><li><span><a href="#Сводная-результатов-обучения" data-toc-modified-id="Сводная-результатов-обучения-1.2.5"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>Сводная результатов обучения</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Classic-ML" data-toc-modified-id="Classic-ML-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Classic ML</a></span><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-LGBMClassifier" data-toc-modified-id="Модель-LGBMClassifier-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Модель LGBMClassifier</a></span></li><li><span><a href="#Модель-RandomForestClassifier" data-toc-modified-id="Модель-RandomForestClassifier-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Модель RandomForestClassifier</a></span></li><li><span><a href="#Тестирование-наилучшей-модели" data-toc-modified-id="Тестирование-наилучшей-модели-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Тестирование наилучшей модели</a></span></li><li><span><a href="#Сводная-результатов-обучения" data-toc-modified-id="Сводная-результатов-обучения-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>Сводная результатов обучения</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

In [1]:
# pip install protobuf==3.20.*

In [4]:
### импорт библиотек
import numpy as np
import pandas as pd
import torch
import re
import time
from transformers import BertTokenizer, BertForMaskedLM
from tqdm import notebook
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm, tqdm_notebook
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import compute_class_weight

## BERT

Использована модель из следующего источника:
- https://huggingface.co/bert-base-cased/tree/main
- BERT-base, English, cased, 12-layer, 768-hidden, 12-heads, 110M parameters

### Подготовка

In [5]:
# откроем файл
data = pd.read_csv('toxic_comments.csv', index_col=0)
# перезапишем файл для работы в рамках озу
data = data.sample(512, random_state=2023).reset_index(drop=True)
# выведем первые пять строк
data.head(5)

,text,toxic
0,This is ridiculous. You yourself don't think ...,1
1,Linking to fan sites\nYou have done a lot of g...,0
2,Good work! \n\nDid you know that this article ...,0
3,"Pundit, are you Montesquio?",0
4,It's till not clear what you're proposing. It ...,0


In [6]:
# опеределим предобученные токенайзер и модель
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")
model = BertForMaskedLM.from_pretrained("unitary/toxic-bert")

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
You should probably TRAIN t

In [7]:
# обозначим модуль для пандас, чтобы устранить ошибку
notebook.tqdm().pandas()
# обрежем токенайзер под нужное количество токенов
tokenized = data['text'].progress_apply(lambda x: tokenizer.encode(x, max_length=128, truncation=True, add_special_tokens=True))
# в токенайзере дополним пропуски нулями
padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True)
# преобразуем маску
attention_mask = padded > 0
attention_mask = attention_mask.type(torch.LongTensor)

0it [00:00, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

In [8]:
# создаем датасет
dataset = TensorDataset(attention_mask, padded)
# создаем загрузчик данных с батчом 32
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)
# выведем информацию о доступных cpu или gpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(f'Device: {device}')

Device: cpu


In [9]:
%%time
# пустой список для хранения эмбеддингов
embeddings = []
# определим модель с учетом доступного девайса
model.to(device)
model.eval()
# воспользуемся библиотекой tqdm для визуализации прогресса
for attention_mask, padded in notebook.tqdm(dataloader):
    attention_mask, padded = attention_mask.to(device), padded.to(device)
    # укажем что градиенты не требуются
    with torch.no_grad():
        # получим эмбеддинги для батча, передав модели данные и маску
        batch_embeddings = model(padded, attention_mask=attention_mask)
    # преобразуем элементы методом numpy() к типу numpy.array
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/16 [00:00<?, ?it/s]

CPU times: total: 16min 19s
Wall time: 4min 56s


In [10]:
# %%time
# # обучать будем небольшими батчами, чтобы не сломать ядро
# batch_size = 5
# # пустой список для хранения эмбеддингов
# embeddings = []
# for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
#         # преобразуем данные
#         batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
#         # преобразуем маску
#         attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
#         # укажем что градиенты не требуются
#         with torch.no_grad():
#             # получим эмбеддинги для батча, передав модели данные и маску
#             batch_embeddings = model(batch, attention_mask=attention_mask_batch)
#         # преобразуем элементы методом numpy() к типу numpy.array
#         embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

### Обучение

In [11]:
# обозначим признаки и цели
features = np.concatenate(embeddings)
target = data['toxic']
# разделим данные на выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, shuffle=True, random_state=42)
# выведем получившиеся размерности
display(features_train.shape)
display(target_train.shape)
display(features_test.shape)
display(target_test.shape)

(358, 30522)

(358,)

(154, 30522)

(154,)

#### Модель LogisticRegression

In [12]:
%%time
start_11 = time.time()
### обучим модель LogisticRegression с помощью GridSearchCV для расчета f1
# параметры по сетке
parameters_11 = {
    'solver': ['liblinear', 'saga'],
    'C': [2, 10]
}
log_reg11 = LogisticRegression(random_state=42, class_weight='balanced')
clf_11 = GridSearchCV(log_reg11, parameters_11, scoring='f1', cv=3, n_jobs=1, verbose=2)
clf_11.fit(features_train, target_train)
score_11 = clf_11.best_score_
display(score_11)
display(clf_11.best_params_)
end_11 = time.time() - start_11

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ..............................C=2, solver=liblinear; total time=   2.4s
[CV] END ..............................C=2, solver=liblinear; total time=   2.2s
[CV] END ..............................C=2, solver=liblinear; total time=   2.4s


C:\Users\Spline\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV] END ...................................C=2, solver=saga; total time=  12.4s


C:\Users\Spline\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV] END ...................................C=2, solver=saga; total time=  12.5s


...

[CV] END ..................................C=10, solver=saga; total time=  12.7s


C:\Users\Spline\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


0.8627582982702197

{'C': 2, 'solver': 'saga'}

CPU times: total: 1min 49s
Wall time: 1min 49s


#### Модель LGBMClassifier

In [13]:
### создадим баланс классов
classes = np.unique(target_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=target_train)
class_weights = dict(zip(classes, weights))

In [14]:
%%time
start_22 = time.time()

### обучим модель LightGBM с помощью GridSearchCV для расчета f1 меры
## параметры по сетке
parameters_22 = {
    'n_estimators' : [10, 30],
    'learning_rate' : [0.15, 0.1]
}
lgb_clf22 = LGBMClassifier(random_state=42, class_weight='balanced')
clf_22 = GridSearchCV(lgb_clf22, parameters_22, scoring='f1', cv=3, n_jobs=1, verbose=2)
clf_22.fit(features_train, target_train)
score_22 = clf_22.best_score_
display(score_22)
display(clf_22.best_params_)
end_22 = time.time() - start_22

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ................learning_rate=0.15, n_estimators=10; total time=   7.4s
[CV] END ................learning_rate=0.15, n_estimators=10; total time=   6.4s
[CV] END ................learning_rate=0.15, n_estimators=10; total time=   7.1s
[CV] END ................learning_rate=0.15, n_estimators=30; total time=  11.2s
[CV] END ................learning_rate=0.15, n_estimators=30; total time=  10.3s
[CV] END ................learning_rate=0.15, n_estimators=30; total time=  11.0s
[CV] END .................learning_rate=0.1, n_estimators=10; total time=   7.0s
[CV] END .................learning_rate=0.1, n_estimators=10; total time=   6.5s
[CV] END .................learning_rate=0.1, n_estimators=10; total time=   6.9s
[CV] END .................learning_rate=0.1, n_estimators=30; total time=  10.9s
[CV] END .................learning_rate=0.1, n_estimators=30; total time=   9.8s
[CV] END .................learning_rate=0.1, n_es

0.8549494949494948

{'learning_rate': 0.15, 'n_estimators': 30}

CPU times: total: 9min 26s
Wall time: 2min 4s


#### Модель RandomForestClassifier

In [15]:
%%time
start_33 = time.time()
### обучим модель RandomForestClassifier с помощью GridSearchCV для расчета f1 меры
# параметры по сетке
parameters_33 = {
    'n_estimators': [100, 200],
    'max_depth': [10, 50],
}
ran_for33 = RandomForestClassifier(random_state=42, class_weight='balanced')
clf_33 = GridSearchCV(ran_for33, parameters_33, scoring='f1', cv=3, n_jobs=1, verbose=2)
clf_33.fit(features_train, target_train)
score_33 = clf_33.best_score_
display(score_33)
display(clf_33.best_params_)
end_33 = time.time() - start_33

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.6s
[CV] END .....................max_depth=10, n_estimators=200; total time=   1.5s
[CV] END .....................max_depth=10, n_estimators=200; total time=   1.5s
[CV] END .....................max_depth=10, n_estimators=200; total time=   1.3s
[CV] END .....................max_depth=50, n_estimators=100; total time=   0.6s
[CV] END .....................max_depth=50, n_estimators=100; total time=   0.6s
[CV] END .....................max_depth=50, n_estimators=100; total time=   0.6s
[CV] END .....................max_depth=50, n_estimators=200; total time=   1.3s
[CV] END .....................max_depth=50, n_estimators=200; total time=   1.3s
[CV] END .....................max_depth=50, n_est

0.8892339544513458

{'max_depth': 10, 'n_estimators': 100}

CPU times: total: 14.1 s
Wall time: 14.1 s


#### Тестирование наилучшей модели

In [27]:
%%time
# проверим результаты на тестовой выборке с подобранными гиперпараметрами
ran_for_bert_test = RandomForestClassifier(random_state=42, class_weight='balanced', max_depth=10, n_estimators=100)
ran_for_bert_test.fit(features_train, target_train)
predictions_bert_test= ran_for_bert_test.predict(features_test)
predictions_bert_test = pd.DataFrame(predictions_bert_test)
score_bert_test = f1_score(target_test, predictions_bert_test)
display(score_bert_test)

0.9411764705882353

CPU times: total: 719 ms
Wall time: 707 ms


#### Сводная результатов обучения

In [28]:
### выведем результаты как таблицу с данными
total_row_00 = ['f1', score_11, score_22, score_33]
total_row_11 = ['time_learn, min', end_11/60, end_22/60, end_33/60]
df_temp_scores1 = pd.DataFrame([total_row_00, total_row_11])
df_temp_scores1.columns = ['Metrics', 'LogisticRegression', 'LGBMClassifier', 'RandomForestClassifier']
display(df_temp_scores1.set_index('Metrics'))

,LogisticRegression,LGBMClassifier,RandomForestClassifier
Metrics,,,
f1,0.862758,0.854949,0.889234
"time_learn, min",1.830811,2.070883,0.234567


### Выводы

- При обучении модели Логистической регрессии результат для метрики `f1` равен `0.862`.
- При обучении модели LGBM результат для метрики `f1` равен `0.854`.
- При обучении модели Случайного леса результат для метрики `f1` равен `0.889`.
- Лучшей можно признать модель Случайного леса с самым быстрым временем обучения.
- На тесте лучшей модели был получен результат метрики `f1` равный `0.941`

## Classic ML

### Подготовка

In [18]:
# откроем файл
df = pd.read_csv('toxic_comments.csv', index_col=0)
# выведем первые пять строк
display(df.head(5))
# выведем размер таблицы
display(df.shape)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


(159292, 2)

In [19]:
%%time
# добавим недостающие разделы
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
# создадим класс для лемматизации
lemm = WordNetLemmatizer()
# создадим собственную функцию
def lemmatize(text):
    t = re.sub(re.compile('<.*?>,'), '', text) # убираем значки
    t = re.sub('[^A-Za-z0-9]+', ' ', t) # оставляем слова
    t = t.lower() # изменяем регистр на строчной
    tokens = nltk.word_tokenize(t) # создаем токены
    lem = [lemm.lemmatize(t) for t in tokens] # создаем леммы
    lem = ' '.join(lem) # соединяем элементы списка
    return lem
# применим функцию лемматизации
df['lemm_text'] = df['text'].apply(lambda x:lemmatize(x))
# выведем результат
display(df.head(5))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Spline\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Spline\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Spline\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


CPU times: total: 2min 9s
Wall time: 2min 10s


In [20]:
# обозначим признаки и цели
features_ml = df['lemm_text']
target_ml = df['toxic']
# деление на обучающую и тестовую выборки
features_train_ml, features_test_ml, target_train_ml, target_test_ml = train_test_split(
    features_ml, target_ml, test_size=0.3, stratify=target_ml, random_state=42)
display('Train:', features_train_ml.shape, target_train_ml.shape)
display('Test:', features_test_ml.shape, target_test_ml.shape)

'Train:'

(111504,)

(111504,)

'Test:'

(47788,)

(47788,)

In [21]:
# обозначим корпусы текстов для выборок
corpus_train = features_train_ml.copy()
corpus_test = features_test_ml.copy()
# подключим стоп-слова
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
# создадим счетчик
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
# посчитаем слова для корпуса текстов, для каждой выборки по отдельности
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)
# выведем результаты на экран
display("Размер матрицы для обучения:", tf_idf_train.shape)
display("Размер матрицы для теста:", tf_idf_test.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Spline\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'Размер матрицы для обучения:'

(111504, 138131)

'Размер матрицы для теста:'

(47788, 138131)

### Обучение

#### Модель LogisticRegression

In [22]:
%%time
start_1 = time.time()
### обучим модель LogisticRegression с помощью GridSearchCV для расчета f1
# параметры по сетке
parameters_1 = {
    'max_iter': [100, 1000],
    'C': [1e1, 1e2],
    'solver': ['newton-cg', 'saga']
}
log_reg1 = LogisticRegression(random_state=42, class_weight='balanced')
clf_1 = GridSearchCV(log_reg1, parameters_1, scoring='f1', cv=3, n_jobs=1, verbose=2)
clf_1.fit(tf_idf_train, target_train_ml)
score_1 = clf_1.best_score_
display(score_1)
display(clf_1.best_params_)
end_1 = time.time() - start_1

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END .............C=10.0, max_iter=100, solver=newton-cg; total time=   4.6s
[CV] END .............C=10.0, max_iter=100, solver=newton-cg; total time=   4.3s
[CV] END .............C=10.0, max_iter=100, solver=newton-cg; total time=   4.5s


C:\Users\Spline\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV] END ..................C=10.0, max_iter=100, solver=saga; total time=   9.0s


C:\Users\Spline\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV] END ..................C=10.0, max_iter=100, solver=saga; total time=   8.7s


...

C:\Users\Spline\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


[CV] END ................C=100.0, max_iter=1000, solver=saga; total time= 1.5min


0.76412154076785

{'C': 10.0, 'max_iter': 100, 'solver': 'newton-cg'}

CPU times: total: 11min 21s
Wall time: 11min 22s


#### Модель LGBMClassifier

In [23]:
%%time
start_2 = time.time()

### обучим модель LightGBM с помощью GridSearchCV для расчета f1 меры
## параметры по сетке
parameters_2 = {
    'n_estimators' : [10, 30],
    'learning_rate' : [0.15, 0.1]
}
lgb_clf2 = LGBMClassifier(random_state=42, class_weight='balanced')
clf_2 = GridSearchCV(lgb_clf2, parameters_2, scoring='f1', cv=3, n_jobs=1, verbose=2)
clf_2.fit(features_train, target_train)
score_2 = clf_2.best_score_
display(score_2)
display(clf_2.best_params_)
end_2 = time.time() - start_2

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ................learning_rate=0.15, n_estimators=10; total time=   7.1s
[CV] END ................learning_rate=0.15, n_estimators=10; total time=   6.7s
[CV] END ................learning_rate=0.15, n_estimators=10; total time=   7.1s
[CV] END ................learning_rate=0.15, n_estimators=30; total time=  11.7s
[CV] END ................learning_rate=0.15, n_estimators=30; total time=  10.4s
[CV] END ................learning_rate=0.15, n_estimators=30; total time=  10.2s
[CV] END .................learning_rate=0.1, n_estimators=10; total time=   7.3s
[CV] END .................learning_rate=0.1, n_estimators=10; total time=   6.2s
[CV] END .................learning_rate=0.1, n_estimators=10; total time=   7.1s
[CV] END .................learning_rate=0.1, n_estimators=30; total time=  10.2s
[CV] END .................learning_rate=0.1, n_estimators=30; total time=  10.4s
[CV] END .................learning_rate=0.1, n_es

0.8549494949494948

{'learning_rate': 0.15, 'n_estimators': 30}

CPU times: total: 9min 21s
Wall time: 2min 3s


#### Модель RandomForestClassifier

In [24]:
%%time
start_3 = time.time()
### обучим модель RandomForestClassifier с помощью GridSearchCV для расчета f1 меры
# параметры по сетке
parameters_3 = {
    'n_estimators': [100, 200],
    'max_depth': [10, 50],
    'min_samples_leaf': [5, 10]
}
ran_for3 = RandomForestClassifier(random_state=42, class_weight='balanced')
clf_3 = GridSearchCV(ran_for3, parameters_3, scoring='f1', cv=3, n_jobs=1, verbose=2)
clf_3.fit(tf_idf_train, target_train_ml)
score_3 = clf_3.best_score_
display(score_3)
display(clf_3.best_params_)
end_3 = time.time() - start_3

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END .max_depth=10, min_samples_leaf=5, n_estimators=100; total time=  19.4s
[CV] END .max_depth=10, min_samples_leaf=5, n_estimators=100; total time=  19.4s
[CV] END .max_depth=10, min_samples_leaf=5, n_estimators=100; total time=  20.3s
[CV] END .max_depth=10, min_samples_leaf=5, n_estimators=200; total time=  40.7s
[CV] END .max_depth=10, min_samples_leaf=5, n_estimators=200; total time=  38.8s
[CV] END .max_depth=10, min_samples_leaf=5, n_estimators=200; total time=  39.3s
[CV] END max_depth=10, min_samples_leaf=10, n_estimators=100; total time=  19.1s
[CV] END max_depth=10, min_samples_leaf=10, n_estimators=100; total time=  19.2s
[CV] END max_depth=10, min_samples_leaf=10, n_estimators=100; total time=  19.3s
[CV] END max_depth=10, min_samples_leaf=10, n_estimators=200; total time=  38.6s
[CV] END max_depth=10, min_samples_leaf=10, n_estimators=200; total time=  38.1s
[CV] END max_depth=10, min_samples_leaf=10, n_est

0.46133490238640334

{'max_depth': 50, 'min_samples_leaf': 5, 'n_estimators': 200}

CPU times: total: 29min 27s
Wall time: 29min 29s


#### Тестирование наилучшей модели

In [29]:
%%time
# проверим результаты на тестовой выборке с подобранными гиперпараметрами
lgb_clf = LGBMClassifier(random_state=42, class_weight='balanced', learning_rate=0.15, n_estimators=30)
lgb_clf.fit(tf_idf_train, target_train_ml)
predictions_test= lgb_clf.predict(tf_idf_test)
predictions_test = pd.DataFrame(predictions_test)
score_test = f1_score(target_test_ml, predictions_test)
display(score_test)

0.7142034548944337

CPU times: total: 1min 19s
Wall time: 11.9 s


#### Сводная результатов обучения

In [30]:
### выведем результаты как таблицу с данными
total_row_0 = ['f1', score_1, score_2, score_3]
total_row_1 = ['time_learn, min', end_1/60, end_2/60, end_3/60]
df_temp_scores = pd.DataFrame([total_row_0, total_row_1])
df_temp_scores.columns = ['Metrics', 'LogisticRegression', 'LGBMClassifier', 'RandomForestClassifier']
display(df_temp_scores.set_index('Metrics'))

,LogisticRegression,LGBMClassifier,RandomForestClassifier
Metrics,,,
f1,0.764122,0.854949,0.461335
"time_learn, min",11.368879,2.050535,29.498001


### Выводы

- При обучении модели Логистической регрессии результат для метрики `f1` равен `0.764`.
- При обучении модели LGBM результат для метрики `f1` равен `0.854`.
- При обучении модели Случайного леса результат для метрики `f1` равен `0.461`.
- Лучшей можно признать модель LGBM с самым быстрым временем обучения.
- На тесте лучшей модели был получен результат метрики `f1` равный `0.714`